In [10]:
import pymongo
import numpy as np
import json
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

In [11]:
import os
os.environ['JAVA_HOME']='C:\Program Files\Java'
os.environ['PYSPARK_SUBMIT_ARGS']="--master local[2] pyspark-shell"

In [12]:
client = MongoClient('mongodb://ec2-3-144-232-39.us-east-2.compute.amazonaws.com:8018')
 


In [13]:
client

MongoClient(host=['ec2-3-144-232-39.us-east-2.compute.amazonaws.com:8018'], document_class=dict, tz_aware=False, connect=True)

In [7]:
db_list = client.list_database_names()
# Iterate over each database
for db_name in db_list:
    # Get the database object
    db = client[db_name]
    # Get the list of collections in the database
    coll_list = db.list_collection_names()
    # Print the collection names
    print(f"Collection names in {db_name}: {coll_list}")

Collection names in admin: ['system.users', 'system.version']
Collection names in config: ['system.sessions']
Collection names in local: ['startup_log']
Collection names in movie_system: ['movie_writer', 'user_review_movie', 'movie_movie_directors', 'movie_genre', 'movie_director', 'user_info', 'movie_movie_genres', 'movie_movie_actors', 'user_rate_movie', 'movie_movie', 'movie_actor', 'movie_movie_writers']
Collection names in test: ['book']


In [30]:
collection = client['movie_system']["movie_movie"]
# print the first five documents in the collection
for document in collection.find().limit(5):
    print(document)

{'_id': ObjectId('644c12e87ba2112c40e5019e'), 'id': 89323, 'IMDB_id': '0002461', 'name': 'Richard III', 'year': '1912', 'numVotes': '297', 'runtimeMinutes': '55', 'brief_summary': 'Richard of Gloucester uses manipulation and murder to gain the English throne.', 'avgRating': '5.7', 'review': ['Besides its historical importance, this silent screen adaptation of Shakespeare\'s "Richard III" is a pretty good movie in its own right.  It has several good features that compensate for the lack of dialogue and the other cinematic limitations of the time.  The result is something a bit different from watching the play, but still a good story that does retain much of the emphasis of the original.The adaptation is noteworthy for the liveliness and the detail that went into most of the scenes.  They also made generally good decisions in adapting the story, by high-lighting the parts that would work most effectively on film without dialogue, and also in filming some events that were not actually in 

In [5]:
collection = client['movie_system']["movie_movie"]
documents = collection.find({}, {"avgRating": 1})
documents.sort("avgRating", pymongo.DESCENDING)
for document in documents:
    print(document['avgRating'])

9.3
9.2
9.2
9.0
9.0
9.0
9.0
9.0
8.9
8.9
8.9
8.9
8.9
8.8
8.8
8.8
8.8
8.8
8.8
8.8
8.8
8.8
8.8
8.8
8.8
8.7
8.7
8.7
8.7
8.7
8.7
8.7
8.7
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.6
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.5
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.4
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.3
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2
8.2


In [26]:
total_docs = collection.count_documents({})
print(total_docs)

1


In [38]:
documents = collection.find({}, {"id": 1})
movieid_list = []
# Print the "id" of each document
for document in documents:
    movieid_list.append(document['id'])
    print(document['id'])

89323
100946
82007
86518
90339
66284
74097
111901
92883
103424
84852
86471
80725
76758
7065
25735
113223
91503
73056
92132
69509
62383
121415
73480
126991
72315
92691
7243
91562
72921
101375
48374
96706
78629
126993
82332
92231
107632
71705
100054
82390
110512
69233
6988
97821
3132
89977
84838
117905
109640
78690
6987
2821
2823
72674
90907
47131
25736
25737
57482
126997
25738
59945
60145
3231
84836
128681
7077
53853
45707
78057
63007
80144
94941
43822
112056
3310
25741
79278
25742
6499
111885
81435
25743
59954
47501
46199
86664
65227
4768
120208
7288
110048
25744
25745
8038
117482
44587
1348
88278
56626
47129
3195
72344
110409
72854
25746
8609
71187
78376
8235
88591
77433
3140
84176
122397
25747
3641
71999
106188
116439
72363
81432
49668
25748
57480
107194
4405
25749
47135
109062
7074
72001
45194
45192
74164
97319
124490
25750
68173
7302
106466
112471
98751
25751
3742
98213
73158
2777
63692
111347
104480
25752
88638
3133
3629
120787
25753
73125
87745
96962
91286
4277
85205
68899
78093


In [54]:
import random
collection = client['movie_system']['user_rate_movie']

# Get the "movie_movie" collection
movie_collection = client['movie_system']['movie_movie']

# Get the number of movies
num_movies = movie_collection.count_documents({})
doc_id = 1
for user_id in range(5):
    for movie in movie_collection.find().limit(5):
        doc = {
            "id": doc_id,
            "user_id": user_id,
            "movie_id": movie["id"],
            "rating": random.randint(0, 10)
        }
        collection.insert_one(doc)
        doc_id += 1

In [52]:
#collection.drop()

In [56]:
for document in collection.find():
    print(document)

{'_id': ObjectId('644c7c33146be553f2109dd1'), 'id': 1, 'user_id': 0, 'movie_id': 89323, 'rating': 1}
{'_id': ObjectId('644c7c33146be553f2109dd2'), 'id': 2, 'user_id': 0, 'movie_id': 100946, 'rating': 9}
{'_id': ObjectId('644c7c33146be553f2109dd3'), 'id': 3, 'user_id': 0, 'movie_id': 82007, 'rating': 10}
{'_id': ObjectId('644c7c33146be553f2109dd4'), 'id': 4, 'user_id': 0, 'movie_id': 86518, 'rating': 6}
{'_id': ObjectId('644c7c33146be553f2109dd5'), 'id': 5, 'user_id': 0, 'movie_id': 90339, 'rating': 6}
{'_id': ObjectId('644c7c33146be553f2109dd6'), 'id': 6, 'user_id': 1, 'movie_id': 89323, 'rating': 8}
{'_id': ObjectId('644c7c33146be553f2109dd7'), 'id': 7, 'user_id': 1, 'movie_id': 100946, 'rating': 5}
{'_id': ObjectId('644c7c33146be553f2109dd8'), 'id': 8, 'user_id': 1, 'movie_id': 82007, 'rating': 10}
{'_id': ObjectId('644c7c33146be553f2109dd9'), 'id': 9, 'user_id': 1, 'movie_id': 86518, 'rating': 2}
{'_id': ObjectId('644c7c33146be553f2109dda'), 'id': 10, 'user_id': 1, 'movie_id': 90339

In [ ]:
from pyspark.sql.functions import col
from pyspark import SparkContext, SparkConf
# Create a PySpark session
#spark = SparkSession.builder.appName("ALS Recommendation").getOrCreate()
cf=SparkConf()
cf.set("spark.submit.deployMode", "client")
sc=SparkContext.getOrCreate(cf)
sc

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ALS Recommendation').config("spark.some.config.option","some-value").getOrCreate()


In [6]:

# Read the "user_rate_movie" collection from MongoDB
df = spark.read.format("mongo").option("uri", "mongodb://ec2-3-144-232-39.us-east-2.compute.amazonaws.com:8018/movie_system.user_rate_movie").load()

# Split the data into training and test sets
(training_data, test_data) = df.randomSplit([0.8, 0.2])

NameError: name 'spark' is not defined

In [ ]:
from pyspark.ml.recommendation import ALS

# Train the ALS model on the training data
als = ALS(userCol="user_id", itemCol="movie_id", ratingCol="rating", rank=10)
model = als.fit(training_data)


In [ ]:
# Generate recommendations for all users in the test set
user_recommendations = model.recommendForAllUsers(30)
# Filter the recommendations for a given user ID and extract the movie IDs
user_id = 1
top_movie_ids = [r[0] for r in user_recommendations.filter(col("user_id") == user_id).collect()[0]["recommendations"]]
# Print the top 30 movie IDs for the given user ID
print(top_movie_ids)
